# Merge single cells from CellProfiler outputs using CytoTable

In [1]:
import argparse
import pathlib
import pprint
import sys

import pandas as pd
from cytotable import convert, presets

sys.path.append("../../../utils")
import sc_extraction_utils as sc_utils
from parsl.config import Config
from parsl.executors import HighThroughputExecutor

## Set paths and variables

All paths must be string but we use pathlib to show which variables are paths

In [2]:
# check if in a jupyter notebook
try:
    cfg = get_ipython().config
    in_notebook = True
except NameError:
    in_notebook = False

if not in_notebook:
    print("Running as script")
    # set up arg parser
    parser = argparse.ArgumentParser(description="Segment the nuclei of a tiff image")

    parser.add_argument(
        "--input_dir",
        type=str,
        help="Path to the input directory containing the tiff images",
    )

    args = parser.parse_args()
    input_dir = pathlib.Path(args.input_dir).resolve(strict=True)
else:
    print("Running in a notebook")
    input_dir = pathlib.Path(
        "../../3.cellprofiling/analysis_output/W0052_F0001"
    ).resolve(strict=True)

Running in a notebook


In [3]:
# type of file output from CytoTable (currently only parquet)
dest_datatype = "parquet"

# directory where parquet files are saved to
output_dir = pathlib.Path("../data/converted_data")
output_dir.mkdir(exist_ok=True, parents=True)

In [4]:
# get the .sqlite file from the input directory
sqlite_file = list(input_dir.glob("*.sqlite"))[0]
dest_path = output_dir / str(sqlite_file.parent).split("/")[-1]
dest_path.mkdir(exist_ok=True, parents=True)
dest_path = dest_path / f"{sqlite_file.stem}.{dest_datatype}"
print(f"Destination path: {dest_path}")

Destination path: ../data/converted_data/W0052_F0001/pyroptosis_timelapse.parquet


## set config joins for each preset

In [5]:
# preset configurations based on typical CellProfiler outputs
preset = "cellprofiler_sqlite_pycytominer"
presets.config[preset][
    "CONFIG_JOINS"
    # remove Image_Metadata_Plate from SELECT as this metadata was not extracted from file names
    # add Image_Metadata_Site as this is an important metadata when finding where single cells are located
] = """WITH Per_Image_Filtered AS (
                SELECT
                    Metadata_ImageNumber,
                    Image_Metadata_Time,
                    Image_Metadata_Well,
                    Image_Metadata_FOV,
                    Image_PathName_CL488,
                    Image_PathName_CL561,
                    Image_PathName_GSDM,
                    Image_PathName_BF,
                    Image_PathName_DNA,
                    Image_FileName_CL488,
                    Image_FileName_CL561,
                    Image_FileName_GSDM,
                    Image_FileName_BF,
                    Image_FileName_DNA,
                FROM
                    read_parquet('per_image.parquet')
                )
            SELECT
                *
            FROM
                Per_Image_Filtered AS per_image
            LEFT JOIN read_parquet('per_cytoplasm.parquet') AS per_cytoplasm ON
                per_cytoplasm.Metadata_ImageNumber = per_image.Metadata_ImageNumber
            LEFT JOIN read_parquet('per_cells.parquet') AS per_cells ON
                per_cells.Metadata_ImageNumber = per_cytoplasm.Metadata_ImageNumber
                AND per_cells.Metadata_Cells_Number_Object_Number = per_cytoplasm.Metadata_Cytoplasm_Parent_Cells
            LEFT JOIN read_parquet('per_nuclei.parquet') AS per_nuclei ON
                per_nuclei.Metadata_ImageNumber = per_cytoplasm.Metadata_ImageNumber
                AND per_nuclei.Metadata_Nuclei_Number_Object_Number = per_cytoplasm.Metadata_Cytoplasm_Parent_Nuclei
            """

## Convert SQLite file and merge single cell objects into parquet file

This was not run to completion as we use the nbconverted python file for full run.

In [6]:
# merge single cells and output as parquet file
convert(
    source_path=sqlite_file,
    dest_path=dest_path,
    dest_datatype=dest_datatype,
    preset=preset,
    parsl_config=Config(
        executors=[HighThroughputExecutor()],
    ),
    chunk_size=1000,
)

PosixPath('/home/lippincm/Documents/pyroptosis_live-cell_timelapse/Wave1_data/4.processing_profiled_features/data/converted_data/W0052_F0001/pyroptosis_timelapse.parquet')

In [7]:
print(f"Merged and converted {pathlib.Path(dest_path).name}!")
print(f"Saved to {dest_path}")
df = pd.read_parquet(dest_path)
print(f"Shape of {pathlib.Path(dest_path).name}: {df.shape}")
# add single cell count per well as metadata column to parquet file and save back to same path
sc_utils.add_sc_count_metadata_file(
    data_path=dest_path,
    well_column_name="Metadata_ImageNumber",
    file_type="parquet",
)
# read the parquet file to check if metadata was added
df1 = pd.read_parquet(dest_path)
print(f"Shape of {pathlib.Path(dest_path).name}: {df.shape}")
print(f"Added single cell count as metadata to {pathlib.Path(dest_path).name}!")

Merged and converted pyroptosis_timelapse.parquet!
Saved to ../data/converted_data/W0052_F0001/pyroptosis_timelapse.parquet
Shape of pyroptosis_timelapse.parquet: (12384, 2778)
Shape of pyroptosis_timelapse.parquet: (12384, 2778)
Added single cell count as metadata to pyroptosis_timelapse.parquet!


In [8]:
df1.head()

,Metadata_ImageNumber,Image_Metadata_FOV,Metadata_number_of_singlecells,Image_Metadata_Time,Image_Metadata_Well,Metadata_Cells_Number_Object_Number,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_ImageNumber_1,Metadata_ImageNumber_2,...,Nuclei_Texture_Variance_CL561_3_02_256,Nuclei_Texture_Variance_CL561_3_03_256,Nuclei_Texture_Variance_DNA_3_00_256,Nuclei_Texture_Variance_DNA_3_01_256,Nuclei_Texture_Variance_DNA_3_02_256,Nuclei_Texture_Variance_DNA_3_03_256,Nuclei_Texture_Variance_GSDM_3_00_256,Nuclei_Texture_Variance_GSDM_3_01_256,Nuclei_Texture_Variance_GSDM_3_02_256,Nuclei_Texture_Variance_GSDM_3_03_256
0,1,0001,502,00,0052,1.0,1,1,1,1.0,...,0.000000,0.000000,0.184131,0.188453,0.177188,0.181128,0.103136,0.115436,0.113431,0.134241
1,1,0001,502,00,0052,2.0,2,17,1,1.0,...,0.345488,0.667641,0.212552,0.217013,0.210907,0.213728,0.410092,0.382479,0.408034,0.422193
2,1,0001,502,00,0052,3.0,3,2,1,1.0,...,0.000000,0.000000,0.232724,0.223048,0.226669,0.223174,0.090572,0.087780,0.096994,0.087674
3,1,0001,502,00,0052,4.0,4,3,1,1.0,...,0.000000,0.000000,0.437401,0.436013,0.421019,0.423367,0.000000,0.000000,0.000000,0.000000
4,1,0001,502,00,0052,5.0,5,32,1,1.0,...,0.016125,0.000000,0.473553,0.457916,0.465556,0.472717,0.000000,0.000000,0.000000,0.000000
